In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import re
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances
# key => 
# 산업안전보건법 + "1"
# 건설기술진흥법 + "2"
# 위험물안전관리법 + '3'
# 전기안전관리법 + "4"
# 화학물질관리법 + "5"

In [2]:
df_industrial = pd.read_excel("산업안전보건법.xlsx")
df_erection = pd.read_excel("건설기술진흥법.xlsx")
df_danger = pd.read_excel("위험물안전관리법.xlsx")
df_electric = pd.read_excel("전기안전관리법.xlsx")
df_chemistry = pd.read_excel("화학물질관리법.xlsx")
df_info = pd.DataFrame()
df_info = df_info.append(df_industrial,ignore_index = True)
df_info = df_info.append(df_erection,ignore_index = True)
df_info = df_info.append(df_danger,ignore_index = True)
df_info = df_info.append(df_electric,ignore_index = True)
df_info = df_info.append(df_chemistry,ignore_index = True)
df_info

,키,조내용
0,10011,제1조(목적) 이 법은 산업 안전 및 보건에 관한 기준을 확립하고 그 책임의 소재를...
1,20011,제2조(정의) 이 법에서 사용하는 용어의 뜻은 다음과 같다. <개정 2020.5.2...
2,30011,"제3조(적용 범위) 이 법은 모든 사업에 적용한다. 다만, 유해ㆍ위험의 정도, 사업..."
3,40011,제4조(정부의 책무) 정부는 이 법의 목적을 달성하기 위하여 다음 각 호의 사항을 ...
4,50011,제5조(사업주 등의 의무) 사업주(제77조에 따른 특수형태근로종사자로부터 노무를 제...
...,...,...
452,600015,제60조(벌칙) 제43조제2항에 따라 즉시 신고를 하지 아니한 자는 2년 이하의 징...
453,610015,제61조(벌칙) 다음 각 호의 어느 하나에 해당하는 자는 1년 이하의 징역 또는 3...
454,620015,제62조(벌칙) 다음 각 호의 어느 하나에 해당하는 자는 6개월 이하의 징역 또는 ...
455,630015,"제63조(양벌규정) 법인의 대표자나 법인 또는 개인의 대리인, 사용인, 그 밖의 종..."


In [3]:
#df_info.to_excel('전체법.xlsx',index = False)

In [4]:
# 불용어 리스트
# 불용어, 불필요 단어 제거
stop_words_df = pd.read_excel("stopwords.xlsx")
stop_words_df

,불용어
0,아
1,휴
2,아이구
3,아이쿠
4,아이고
...,...
695,필요한
696,등을
697,해당하는
698,자가


In [5]:
posts = df_info.get("조내용")
posts_key = df_info.get("키")
key = [i for i in posts_key]
posts

0      제1조(목적) 이 법은 산업 안전 및 보건에 관한 기준을 확립하고 그 책임의 소재를...
1      제2조(정의) 이 법에서 사용하는 용어의 뜻은 다음과 같다. <개정 2020.5.2...
2      제3조(적용 범위) 이 법은 모든 사업에 적용한다. 다만, 유해ㆍ위험의 정도, 사업...
3      제4조(정부의 책무) 정부는 이 법의 목적을 달성하기 위하여 다음 각 호의 사항을 ...
4      제5조(사업주 등의 의무) 사업주(제77조에 따른 특수형태근로종사자로부터 노무를 제...
                             ...                        
452    제60조(벌칙) 제43조제2항에 따라 즉시 신고를 하지 아니한 자는 2년 이하의 징...
453    제61조(벌칙) 다음 각 호의 어느 하나에 해당하는 자는 1년 이하의 징역 또는 3...
454    제62조(벌칙) 다음 각 호의 어느 하나에 해당하는 자는 6개월 이하의 징역 또는 ...
455    제63조(양벌규정) 법인의 대표자나 법인 또는 개인의 대리인, 사용인, 그 밖의 종...
456    제64조(과태료) 다음 각 호의 어느 하나에 해당하는 자에게는 1천만원 이하의 과태...
Name: 조내용, Length: 457, dtype: object

In [6]:
def get_key(**key) :
    dic = {}
    for k, y in key.items():
        if k[-1] == "1" :
            s = "산업안전보건법 제 "+k[0:(len(k)-4)]+"조"
        elif k[-1]=="2":
            s = "건설기술진흥법 제 "+k[0:(len(k)-4)]+"조"
        elif k[-1]=="3":
            s = "위험물안전관리법 제 "+k[0:(len(k)-4)]+"조"
        elif k[-1]=="4":
            s = "전기안전관리법 제 "+k[0:(len(k)-4)]+"조"
        elif k[-1]=="5":
            s = "화학물질관리법 제 "+k[0:(len(k)-4)]+"조"     
        if k[-3] != "0":
            s += "의 "+k[-3]   
        dic[s] = y
    return dic   

In [7]:
from konlpy.tag import Okt
okt = Okt()

oo = okt.pos(posts[0],
        norm=True,   # 정규화(normalization)
        stem=True    # 어간추출(stemming)
        )
print(oo)

[('제', 'Noun'), ('1조', 'Number'), ('(', 'Punctuation'), ('목적', 'Noun'), (')', 'Punctuation'), ('이', 'Noun'), ('법', 'Noun'), ('은', 'Josa'), ('산업', 'Noun'), ('안전', 'Noun'), ('및', 'Noun'), ('보건', 'Noun'), ('에', 'Josa'), ('관', 'Noun'), ('한', 'Josa'), ('기준', 'Noun'), ('을', 'Josa'), ('확립', 'Noun'), ('하고', 'Josa'), ('그', 'Noun'), ('책임', 'Noun'), ('의', 'Josa'), ('소재', 'Noun'), ('를', 'Josa'), ('명확하다', 'Adjective'), ('하다', 'Verb'), ('산업', 'Noun'), ('재해', 'Noun'), ('를', 'Josa'), ('예방', 'Noun'), ('하고', 'Josa'), ('쾌적하다', 'Adjective'), ('작업', 'Noun'), ('환경', 'Noun'), ('을', 'Josa'), ('조', 'Modifier'), ('성함', 'Noun'), ('으로써', 'Josa'), ('노무', 'Noun'), ('를', 'Josa'), ('제공', 'Noun'), ('하다', 'Verb'), ('사람', 'Noun'), ('의', 'Josa'), ('안전', 'Noun'), ('및', 'Noun'), ('보건', 'Noun'), ('을', 'Josa'), ('유지', 'Noun'), ('ㆍ', 'Foreign'), ('증진', 'Noun'), ('함', 'Noun'), ('을', 'Josa'), ('목적', 'Noun'), ('으로', 'Josa'), ('하다', 'Verb'), ('.', 'Punctuation'), ('<', 'Punctuation'), ('개정', 'Noun'), ('2020.5', 'Number'), ('.', '

In [8]:
def tokenizer_1(raw_texts, pos=["Noun","Alpha","Verb","Number"], stopword=list(stop_words_df.get("불용어"))):
    p = okt.pos(raw_texts, 
            norm=True,   # 정규화(normalization)
            stem=True    # 어간추출(stemming)
            )
    o = [word for word, tag in p if len(word) > 1 and tag in pos and word not in stopword]
    return(o)

tokenizer_1(posts[80])


['81조',
 '기계',
 '기구',
 '여자',
 '조치',
 '기계',
 '기구',
 '설비',
 '건축물',
 '대여',
 '대여받다',
 '자다',
 '안전',
 '조치',
 '보건',
 '조치']

In [9]:
from konlpy.tag import Mecab
tagger = Mecab()
# from eunjeon import Mecab
# tagger = Mecab()
def tokenizer_2(post):
    nouns = []
  
    for noun in tagger.nouns(post):
        if noun not in list(stop_words_df.get("불용어")):
            nouns.append(noun)
    return nouns

tokenizer_2(posts[0])

['목적',
 '법',
 '안전',
 '보건',
 '기준',
 '확립',
 '책임',
 '소재',
 '재해',
 '예방',
 '작업',
 '환경',
 '조성',
 '노무',
 '제공',
 '사람',
 '안전',
 '보건',
 '유지',
 '목적',
 '개정']

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorize = TfidfVectorizer(
    tokenizer=tokenizer_1, # 문장에 대한 tokenizer (위에 정의한 함수 이용)
    min_df=1,            # 단어가 출현하는 최소 문서의 개수
    sublinear_tf=True    # tf값에 1+log(tf)를 적용하여 tf값이 무한정 커지는 것을 막음
)

X = vectorize.fit_transform(posts)
X.toarray() 

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [11]:
# 모든 조에 해당하는 euclidean distance 를 계산
euclidean_result = []
for x in range(len(df_info)):
    tmp = []
    for y in range(len(df_info)):
        tmp.append(euclidean_distances(X[x], X[y])[0][0])
    euclidean_result.append(tmp)
# euclidean_result

In [12]:
# 모든 조에 해당하는 euclidean distance 결과 값을
# DataFrame 형태로 변환
euclidean_result_df = pd.DataFrame(euclidean_result)
euclidean_result_df

,0,1,2,3,4,5,6,7,8,9,...,447,448,449,450,451,452,453,454,455,456
0,0.000000,1.289757,1.414214,1.175499,1.248485,1.343371,1.361378,1.360351,1.308307,1.350362,...,1.378859,1.391069,1.398336,1.390360,1.380424,1.414214,1.397602,1.409579,1.414214,1.402701
1,1.289757,0.000000,1.327635,1.336567,1.213869,1.300910,1.341292,1.344032,1.355083,1.229740,...,1.387232,1.388835,1.396432,1.372453,1.384828,1.407918,1.383536,1.388266,1.397695,1.378485
2,1.414214,1.327635,0.000000,1.399259,1.365332,1.386404,1.414214,1.380395,1.414214,1.356870,...,1.379108,1.299542,1.400848,1.378431,1.395492,1.395183,1.395197,1.398395,1.414214,1.395249
3,1.175499,1.336567,1.399259,0.000000,1.300796,1.260531,1.345828,1.288677,1.266167,1.359846,...,1.280975,1.321514,1.395897,1.370300,1.360045,1.402729,1.379076,1.400056,1.392232,1.386225
4,1.248485,1.213869,1.365332,1.300796,0.000000,1.174273,1.365394,1.342641,1.331051,1.319417,...,1.391605,1.398829,1.393355,1.321130,1.336976,1.389226,1.336022,1.354460,1.401320,1.384685
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
452,1.414214,1.407918,1.395183,1.402729,1.389226,1.404570,1.408413,1.400643,1.407798,1.378607,...,1.405327,1.377534,1.281977,1.289129,1.346569,0.000000,1.277802,1.309684,1.414214,1.363454
453,1.397602,1.383536,1.395197,1.379076,1.336022,1.369152,1.348739,1.374589,1.381599,1.368932,...,1.401072,1.391729,1.342874,0.977006,1.215813,1.277802,0.000000,1.060126,1.409455,1.074164
454,1.409579,1.388266,1.398395,1.400056,1.354460,1.407796,1.392560,1.388107,1.409943,1.392056,...,1.404550,1.389393,1.363714,1.119819,1.217507,1.309684,1.060126,0.000000,1.373568,1.228051
455,1.414214,1.397695,1.414214,1.392232,1.401320,1.394538,1.414214,1.389004,1.414214,1.414214,...,1.403486,1.394175,1.363075,1.376693,1.357506,1.414214,1.409455,1.373568,0.000000,1.391955


In [13]:
# euclidean distance 값으로 
# 8조와 가장 비슷한 조를 찾기위한 과정
jo_key = 240013
jo = key.index(jo_key)
y_euclidean = euclidean_result_df[jo]
dic = {}
for i in range(len(y_euclidean)):
    dic[str(posts_key[i])] = y_euclidean[i]
dic = get_key(**dic)
#8조와 비슷한 조 찾기 위함
# euclidean distance 를 dic에 씌워 sorting 함(1부터 10위까지)
sorted(dic.items(), key=lambda x: x[1])[0:15]

[('위험물안전관리법 제 24조', 0.0),
 ('위험물안전관리법 제 26조', 0.9962528193000993),
 ('위험물안전관리법 제 2조', 1.0720465406784396),
 ('위험물안전관리법 제 36조', 1.1067808921475752),
 ('위험물안전관리법 제 27조', 1.158871726101327),
 ('위험물안전관리법 제 12조', 1.1777106267754718),
 ('위험물안전관리법 제 6조', 1.1810273180176456),
 ('위험물안전관리법 제 5조', 1.1864040153489592),
 ('위험물안전관리법 제 8조', 1.1973074059486297),
 ('위험물안전관리법 제 15조', 1.1990670671941461),
 ('위험물안전관리법 제 34조의 3', 1.2007233043843943),
 ('위험물안전관리법 제 14조', 1.2186393238134425),
 ('위험물안전관리법 제 1조', 1.2232036016048824),
 ('위험물안전관리법 제 37조', 1.2257903225192814),
 ('위험물안전관리법 제 4조', 1.2260857417279483)]

In [14]:
cosine_result = []
for x in range(len(df_info)):
    tmp = []
    for y in range(len(df_info)):
        tmp.append(cosine_similarity(X[x], X[y])[0][0])
    cosine_result.append(tmp)

In [15]:
# 모든 조에 해당하는 cosine similarity 결과 값을
# DataFrame 형태로 변환
cosine_result_df = pd.DataFrame(cosine_result)
cosine_result_df

,0,1,2,3,4,5,6,7,8,9,...,447,448,449,450,451,452,453,454,455,456
0,1.000000,0.168263,0.000000,0.309101,0.220643,0.097677,0.073324,0.074722,0.144166,0.088261,...,0.049374,0.032464,0.022328,0.033449,0.047215,0.000000,0.023354,0.006543,0.000000,0.016215
1,0.168263,1.000000,0.118693,0.106794,0.263261,0.153817,0.100468,0.096789,0.081876,0.243870,...,0.037794,0.035569,0.024989,0.058187,0.041125,0.008884,0.042914,0.036359,0.023224,0.049890
2,0.000000,0.118693,1.000000,0.021037,0.067934,0.038943,0.000000,0.047255,0.000000,0.079452,...,0.049031,0.155595,0.018813,0.049964,0.026300,0.026732,0.026713,0.022246,0.000000,0.026640
3,0.309101,0.106794,0.021037,1.000000,0.153965,0.205530,0.094374,0.169656,0.198410,0.075409,...,0.179552,0.126800,0.025736,0.061139,0.075139,0.016175,0.049075,0.019921,0.030845,0.039190
4,0.220643,0.263261,0.067934,0.153965,1.000000,0.310542,0.067849,0.098657,0.114152,0.129570,...,0.031717,0.021638,0.029281,0.127308,0.106247,0.035025,0.107522,0.082719,0.018151,0.041324
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
452,0.000000,0.008884,0.026732,0.016175,0.035025,0.013591,0.008186,0.019100,0.009052,0.049721,...,0.012528,0.051200,0.178267,0.169073,0.093376,1.000000,0.183611,0.142364,0.000000,0.070497
453,0.023354,0.042914,0.026713,0.049075,0.107522,0.062712,0.090451,0.055252,0.045592,0.063012,...,0.018498,0.031545,0.098345,0.522730,0.260899,0.183611,1.000000,0.438066,0.006718,0.423086
454,0.006543,0.036359,0.022246,0.019921,0.082719,0.009055,0.030388,0.036580,0.006030,0.031090,...,0.013620,0.034794,0.070142,0.373003,0.258838,0.142364,0.438066,1.000000,0.056655,0.245946
455,0.000000,0.023224,0.000000,0.030845,0.018151,0.027632,0.000000,0.035333,0.000000,0.000000,...,0.015114,0.028139,0.071014,0.052358,0.078588,0.000000,0.006718,0.056655,1.000000,0.031230


In [30]:
# cosine similarity 값으로 
# 8조와 가장 비슷한 조를 찾기위한 과정
jo_key = 750011
jo = key.index(jo_key)
y_cosine = cosine_result_df[jo]
dic ={}
for i in range(len(y_cosine)):
    dic[str(posts_key[i])] = y_cosine[i]
dic = get_key(**dic)
#8조와 비슷한 조 찾기 위함
# euclidean distance 를 dic에 씌워 sorting 함(1부터 10위까지)
sorted(dic.items(), key=lambda x: x[1], reverse = True)[0:25]

[('산업안전보건법 제 75조', 1.0),
 ('산업안전보건법 제 24조', 0.5638763844730781),
 ('산업안전보건법 제 35조', 0.5128234775796429),
 ('산업안전보건법 제 64조', 0.3324774469811759),
 ('산업안전보건법 제 26조', 0.26333824414438),
 ('건설기술진흥법 제 5조', 0.24992281860865403),
 ('화학물질관리법 제 7조', 0.23570970530006574),
 ('산업안전보건법 제 63조', 0.22166811063594005),
 ('건설기술진흥법 제 64조', 0.21773802137150278),
 ('산업안전보건법 제 62조', 0.21603013960529882),
 ('건설기술진흥법 제 68조', 0.199714406550679),
 ('건설기술진흥법 제 84조', 0.19042257442947474),
 ('산업안전보건법 제 68조', 0.1888398686268973),
 ('산업안전보건법 제 72조', 0.18689979409761848),
 ('산업안전보건법 제 49조', 0.18630712623992443),
 ('산업안전보건법 제 55조', 0.18292844124970786),
 ('산업안전보건법 제 10조', 0.17992803635323562),
 ('산업안전보건법 제 13조', 0.17477585913155863),
 ('산업안전보건법 제 125조', 0.17091408632396965),
 ('건설기술진흥법 제 6조', 0.16682338268726477),
 ('산업안전보건법 제 66조', 0.16045780390148234),
 ('전기안전관리법 제 6조', 0.15775821677182233),
 ('산업안전보건법 제 73조', 0.15754725448216017),
 ('산업안전보건법 제 67조', 0.15475242861237665),
 ('산업안전보건법 제 71조', 0.15387829457035856)]

In [17]:
from sklearn.feature_extraction.text import CountVectorizer
countv = CountVectorizer(stop_words = list(stop_words_df.get("불용어")))
sp_mat = countv.fit_transform(posts)
# 희소행렬을 np array로
sp_mat.toarray()

/opt/anaconda3/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:383: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['가서', '같은', '것과', '결과에', '결론을', '관계가', '관련이', '그런', '그럼에도', '그렇게', '그에', '그치지', '김에', '까닭에', '낫다', '년도', '논하지', '누가', '다시', '달려', '대로', '대해', '되는', '되다', '되어', '들면', '들자면', '듯하다', '따르는', '따름이다', '따지지', '때가', '만은', '만이', '많은', '말하면', '말할것도', '몰라도', '몰랏다', '못하다', '못하다하기보다는', '미치다', '바꾸어서', '바꿔', '방면으로', '보면', '보아', '부류의', '비길수', '비추어', '뿐만', '사람들', '상대적으로', '생각이다', '서술한바와같이', '쓰여', '아니다', '아니라', '안다', '안된다', '않고', '않기', '않는다면', '않다', '않다면', '않도록', '않으면', '알겠는가', '어쩔수', '없고', '없다', '예를', '외에', '요만한', '우에', '위에서', '이렇게', '이로', '이르다', '이와', '이유는', '인하여', '임에', '점에서', '정도에', '정도의', '종합한것과같이', '주저하지', '줄은', '지경이다', '틀림없다', '편이', '하고', '하기', '하기만', '하는', '하는것만', '하는것이', '하면', '하지', '한하다', '할수록', '함으로써', '해도', '해서는', '형식으로', '힘이'] not in stop_words.
  warnings.warn('Your stop_words may be inconsis

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [18]:
vectorize = CountVectorizer(
    tokenizer=tokenizer_2, # 문장에 대한 tokenizer (위에 정의한 함수 이용)
    min_df=1,            # 단어가 출현하는 최소 문서의 개수
)

X = vectorize.fit_transform(posts)
X.toarray() 

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [19]:
# 모든 조에 해당하는 euclidean distance 를 계산
euclidean_result = []
for x in range(len(df_info)):
    tmp = []
    for y in range(len(df_info)):
        tmp.append(euclidean_distances(X[x], X[y])[0][0])
    euclidean_result.append(tmp)
# euclidean_result

KeyboardInterrupt: 

In [ ]:
# 모든 조에 해당하는 euclidean distance 결과 값을
# DataFrame 형태로 변환
euclidean_result_df = pd.DataFrame(euclidean_result)
euclidean_result_df

In [ ]:
# euclidean distance 값으로 
# 8조와 가장 비슷한 조를 찾기위한 과정
y_euclidean = euclidean_result_df[jo]
dic = {}
for i in range(len(y_euclidean)):
    dic[str(posts_key[i])] = y_euclidean[i]
dic = get_key(**dic)
#8조와 비슷한 조 찾기 위함
# euclidean distance 를 dic에 씌워 sorting 함(1부터 10위까지)
sorted(dic.items(), key=lambda x: x[1])[0:10]

In [ ]:
cosine_result = []
for x in range(len(df_info)):
    tmp = []
    for y in range(len(df_info)):
        tmp.append(cosine_similarity(X[x], X[y])[0][0])
    cosine_result.append(tmp)

In [ ]:
# 모든 조에 해당하는 cosine similarity 결과 값을
# DataFrame 형태로 변환
cosine_result_df = pd.DataFrame(cosine_result)
cosine_result_df

In [ ]:
# cosine similarity 값으로 
# 8조와 가장 비슷한 조를 찾기위한 과정
y_cosine = cosine_result_df[jo]
dic ={}
for i in range(len(y_cosine)):
    dic[str(posts_key[i])] = y_cosine[i]
dic = get_key(**dic)
#8조와 비슷한 조 찾기 위함
# euclidean distance 를 dic에 씌워 sorting 함(1부터 10위까지)
sorted(dic.items(), key=lambda x: x[1], reverse = True)[0:10]